In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
from PIL import Image

In [6]:
article_tags = pd.read_csv("articles_tags.csv")
article_text = pd.read_csv("../resources/articles_text.csv")
article_tags= article_tags.set_index("Resource URL")
article_text = article_text.set_index("Resource URL")
article_text = article_text[~article_text.index.duplicated(keep='first')]

for i in article_text.index:
    article_text.loc[i, "tech_tag"] =  article_tags.loc[i]["Technical (Y/N)"][0]
    
data = (article_text[(article_text["tech_tag"]=="Y") | (article_text["tech_tag"]=="N")])
data_src = data
data_src = data_src[["Resource Title", "text", "tech_tag"]]
data_src.loc[:, "text"] =  data_src["Resource Title"] + " " +data_src["text"]
data_src.loc[:, "tech_tag"] = data_src["tech_tag"].map({"N": 0, "Y":1})
data_src.drop("Resource Title", axis=1, inplace=True)
data_src = data_src.dropna().reset_index(drop=True)
data=data_src

/opt/conda/lib/python3.8/site-packages/pandas/core/indexing.py:1783: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item_labels[indexer[info_axis]]] = value
/opt/conda/lib/python3.8/site-packages/pandas/core/frame.py:4162: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [4]:
article_tags = pd.read_csv("nn_pred.csv")
article_tags = article_tags.set_index("Resource URL")

data = (article_tags[(article_tags["NN_Pred"]=="Y") | (article_tags["NN_Pred"]=="N")])
data_src = data
data_src = data_src[["Resource Title", "text", "NN_Pred"]]
data_src.loc[:, "text"] =  data_src["Resource Title"] + " " +data_src["text"]
data_src.loc[:, "NN_Pred"] = data_src["NN_Pred"].map({"N": 0, "Y":1})
data_src.drop("Resource Title", axis=1, inplace=True)
data_src = data_src.dropna().reset_index(drop=True)
data=data_src
data["tech_tag"] = data["NN_Pred"]

/opt/conda/lib/python3.8/site-packages/pandas/core/indexing.py:1783: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item_labels[indexer[info_axis]]] = value
/opt/conda/lib/python3.8/site-packages/pandas/core/frame.py:4162: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [8]:
import tensorflow as tf
dataset = tf.data.Dataset.from_tensor_slices((data.text, data.tech_tag))
VALIDATION_SIZE=250
BUFFER_SIZE = 10000
BATCH_SIZE = 64

train_dataset  = dataset.skip(VALIDATION_SIZE).shuffle(BUFFER_SIZE).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
test_dataset  = dataset.take(VALIDATION_SIZE).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

In [9]:
VOCAB_SIZE = 1000
encoder = tf.keras.layers.experimental.preprocessing.TextVectorization(
    max_tokens=VOCAB_SIZE)
encoder.adapt(train_dataset.map(lambda text, label: text))

In [10]:
vocab = np.array(encoder.get_vocabulary())
vocab[:20]

array(['', '[UNK]', 'the', 'of', 'and', 'to', 'in', 'a', 'for', 'with',
       'that', 'covid19', 'is', 'as', 'on', 'are', 'by', 'were', 'we',
       'or'], dtype='<U15')

In [11]:
model = tf.keras.Sequential([
    encoder,
    tf.keras.layers.Embedding(
        input_dim=len(encoder.get_vocabulary()),
        output_dim=64,
        # Use masking to handle the variable sequence lengths
        mask_zero=True),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(2)
])

In [13]:
model.compile(
    optimizer='adam',
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=['accuracy'])

In [14]:
history = model.fit(train_dataset, epochs=10,
                    validation_data=test_dataset,
                    validation_steps=5)

Epoch 1/10
12/12 [==============================] - 18s 674ms/step - loss: 0.6843 - accuracy: 0.6560 - val_loss: 0.6778 - val_accuracy: 0.7520
Epoch 2/10
12/12 [==============================] - 5s 348ms/step - loss: 0.5774 - accuracy: 0.8467
Epoch 3/10
12/12 [==============================] - 5s 426ms/step - loss: 0.4398 - accuracy: 0.7587
Epoch 4/10
12/12 [==============================] - 5s 439ms/step - loss: 0.2607 - accuracy: 0.9040
Epoch 5/10
12/12 [==============================] - 5s 458ms/step - loss: 0.2796 - accuracy: 0.8960
Epoch 6/10
12/12 [==============================] - 5s 457ms/step - loss: 0.1855 - accuracy: 0.9427
Epoch 7/10
12/12 [==============================] - 5s 466ms/step - loss: 0.2176 - accuracy: 0.9120
Epoch 8/10
12/12 [==============================] - 6s 470ms/step - loss: 0.1592 - accuracy: 0.9440
Epoch 9/10
12/12 [==============================] - 5s 425ms/step - loss: 0.1020 - accuracy: 0.9720
Epoch 10/10
12/12 [==============================] - 5s 4

In [16]:
test_loss, test_acc = model.evaluate(test_dataset)

print('Test Loss:', test_loss)
print('Test Accuracy:', test_acc)

4/4 [==============================] - 1s 161ms/step - loss: 0.3891 - accuracy: 0.8640
Test Loss: 0.38905078172683716
Test Accuracy: 0.8640000224113464
